In [1]:
# Deepseek Stuff
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/makemore
# import os
# os.chdir('/content/drive/My Drive/makemore')

/content/drive/MyDrive/makemore


In [9]:
!pip install deepspeed --upgrade --quiet
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.7 MB/s eta 0:00:00


In [10]:
import torch
import deepspeed
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
# train_mlp_ds.py
import torch
import deepspeed
#from mlp_model import MLP


# mlp_model.py
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim=100, hidden_dim=256, output_dim=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# Dummy dataset
def get_data(batch_size=32, input_dim=100, num_samples=1000):
    X = torch.randn(num_samples, input_dim)
    y = torch.randint(0, 10, (num_samples,))
    return torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X, y), batch_size=batch_size
    )

# Loss
criterion = torch.nn.CrossEntropyLoss()

ds_config = {
    "train_batch_size": 32,
    "gradient_accumulation_steps": 1,
    "zero_optimization": {
        "stage": 2
    },
    "fp16": {
        "enabled": False
    }
}

# Load model
model = MLP()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
train_loader = get_data()

# DeepSpeed config
ds_config = "ds_config.json"
model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    optimizer=optimizer,
    config=ds_config
)

# Training loop
for epoch in range(5):
    for batch in train_loader:
        x, y = batch
        x, y = x.to(model_engine.device), y.to(model_engine.device)

        outputs = model_engine(x)
        loss = criterion(outputs, y)

        model_engine.backward(loss)
        model_engine.step()

    print(f"Epoch {epoch} done")

[2025-07-27 17:27:37,704] [INFO] [logging.py:107:log_dist] [Rank 0] DeepSpeed info: version=0.17.2, git-hash=unknown, git-branch=unknown
[2025-07-27 17:27:37,714] [INFO] [config.py:684:__init__] Config mesh_device None world_size = 1
[2025-07-27 17:27:37,721] [INFO] [engine.py:1339:_configure_distributed_model] ********** distributed groups summary **********
	 self.dp_world_size=1
	 self.mp_world_size=1
	 self.seq_dp_world_size=1
	 self.sequence_parallel_size=1
***********************************************
[2025-07-27 17:27:37,724] [INFO] [logging.py:107:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-07-27 17:27:37,726] [INFO] [logging.py:107:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2025-07-27 17:27:37,727] [INFO] [logging.py:107:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-07-27 17:27:37,728] [INFO] [logging.py:107:log_dist] [Rank 0] DeepSpeed Basic Optimizer = AdamW
[2025-07-27 17:27:37,729] [INFO]